# Schedule Vaccination of each employees of the company where each city office can conduct its vaccination drive parallely. Each city will get 200 doses in a day 

In [ ]:
import os

In [2]:
import sys
!{sys.executable} -m pip install findspark

In [3]:
os.environ["SPARK_HOME"]=r"C:\Users\pawan palariya\Downloads\spark-3.1.2-bin-hadoop3.2\spark-3.1.2-bin-hadoop3.2"

In [4]:
os.environ["JAVA_HOME"] =r"C:\Program Files\Java\jdk1.8.0_291"
os.environ["HADOOP_HOME"] =r"C:\Hadoop"

In [5]:
import findspark
findspark.init()

In [6]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
spark = SparkSession.builder.appName('interview_code').getOrCreate()

In [7]:
df_input=spark.read.format("csv").option("header","true").load(r"C:\Users\pawan palariya\Downloads\50000-Records")


In [8]:
df=df_input.withColumn("Full_name", concat_ws(" ",col("First_Name"),col("Middle_Initial"),col("Last_Name")))

In [9]:
df_citywise_employees=df.groupby("city").count().sort(col("count").desc())

In [11]:
list_rows=df_citywise_employees.take(df_citywise_employees.count())

In [17]:
from pyspark.sql.types import StructType,StructField, StringType, IntegerType,DateType
from datetime import date,timedelta
from pyspark.sql import Row
date_today=date.today()+timedelta(days=1)
id=0
rows_list=[]
max_days=0
vaccination_schema = StructType([ \
    StructField("empid",StringType(),True), \
    StructField("empname",StringType(),True), \
    StructField("vaccination_id",StringType(),True), \
    StructField("vaccination_date", DateType(), True), \
    StructField("city", StringType(), True),\
    StructField("vaccination_time", StringType(), True)])
for row in list_rows:
    city=row["city"]
    df_filter_by_city=df.filter(col("city")==city).select("Emp_ID","Full_name")
    city_total_emp=row["count"]
    list_emp_details=df_filter_by_city.take(city_total_emp)
    vaccination_date=date_today
    for i in range(city_total_emp):
        day=i//200
        max_days=max_days if max_days >= day+1 else day+1
        vaccination_date=vaccination_date+timedelta(days=day)
        id=id+1
        vaccination_id="emp_00"+str(id)
        empid=list_emp_details[i]["Emp_ID"]
        empname=list_emp_details[i]["Full_name"]
        if(i%4==0):
            vaccination_time="9am-11am"
        elif(i%4==1):
            vaccination_time="11am-1pm"
        elif(i%4==2):
            vaccination_time="2pm-4pm"
        else:
            vaccination_time="4pm-6pm"
        row=Row(empid=empid,empname=empname,vaccination_id=vaccination_id,vaccination_date=vaccination_date,city=city,vaccination_time=vaccination_time)
        rows_list.append(row)
df=spark.createDataFrame(rows_list,vaccination_schema)


In [22]:
print("total_number of days to vaccinating employees=",max_days)

total_number of days to vaccinating employees= 2


In [20]:
df.count()

50000

In [21]:
df.show(5)

+------+---------------+--------------+----------------+----------+----------------+
| empid|        empname|vaccination_id|vaccination_date|      city|vaccination_time|
+------+---------------+--------------+----------------+----------+----------------+
|188762| Ellis V Mcneel|       emp_001|      2022-06-04|Washington|        9am-11am|
|876349|Collin V Gimbel|       emp_002|      2022-06-04|Washington|        11am-1pm|
|781210|Virgilio R Muro|       emp_003|      2022-06-04|Washington|         2pm-4pm|
|181631| Sheilah D Elam|       emp_004|      2022-06-04|Washington|         4pm-6pm|
|735320|Wilmer Z Carman|       emp_005|      2022-06-04|Washington|        9am-11am|
+------+---------------+--------------+----------------+----------+----------------+
only showing top 5 rows

